In [6]:
import survLime

%load_ext autoreload
%autoreload 2
import os
import sklearn

from sksurv.ensemble import GradientBoostingSurvivalAnalysis, RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis

from sksurv.util import Surv
from sksurv.metrics import integrated_brier_score
from sksurv.svm import FastSurvivalSVM
import pandas as pd

import argparse
import numpy as np

from pycox.models import LogisticHazard
from pycox.evaluation import EvalSurv
from sklearn.model_selection import train_test_split
import sklearn

from sklearn.model_selection import KFold, train_test_split
import xgboost as xgb
import shap

from derma.general.preprocessing.transformers import (TransformToNumeric, 
                                                      TransformToDatetime, 
                                                      ComputeAge,
                                                      TransformToObject,
                                                      KeepColumns,
                                                      ComputeAJCC,
                                                      LinkTumourPartToParent,
                                                      TransformCbRegression,
                                                      ConvertCategoriesToNaN,
                                                      ExponentialTransformer,
                                                      RenameLabValues,
                                                      CustomScaler,
                                                      CustomImputer)

from derma.general.ingestion.data_loader_csv import SurvivalLoader
import config_os as settings_file
from derma.general.preprocessing.encoders import (OrdinalEncoder,
                                                  GenderEncoder,
                                                  AbsentPresentEncoder,
                                                   LABEncoder,
                                                  CategoricalEncoder)
from derma.general.ingestion.data_loader_csv import SurvivalLoader
#from utils.script_utils import (obtain_pipeline, data_preprocessing,
#                            obtain_datasets, obtain_loaders,
#                            obtain_splits, Concordance, obtain_wsi_ids)

import timeit
start_time = timeit.default_timer()
# code you want to evaluate
elapsed = timeit.default_timer() - start_time

np.random.seed(123456)

get_target = lambda df: df[['event','duration']]
path = '/home/carlos.hernandez/datasets/csvs/data-surv_20220302.csv'


clinical_columns = ['patient_gender', 'cutaneous_biopsy_breslow', 'age']

X, _, time, event  = SurvivalLoader('os').load_data(path)
time = [int(x) for x in time]
X['duration'] = time

X['event']    = event
X.dropna(subset=['duration'],inplace=True)
X = X[X['duration']>=0]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
total_c_index  = []
total_ib_score = []

keep_cols = clinical_columns
settings_file.keep_cols = {'cols' : keep_cols}
pipe = sklearn.pipeline.Pipeline(steps=[
    ('TransformToNumeric', TransformToNumeric(**settings_file.transform_to_numeric)), 
    ('TransformToDatetime', TransformToDatetime(**settings_file.transform_to_datetime)),
    ('TransformToObject', TransformToObject(**settings_file.transform_to_object)),
    ('ComputeAge', ComputeAge(**settings_file.compute_age)),
    ('tr_tm', LinkTumourPartToParent(**settings_file.link_tumour_part_to_parent)),
    ('tr_cb', TransformCbRegression(**settings_file.transform_cb_regression)),
    ('tr0', ConvertCategoriesToNaN(**settings_file.convert_categories_to_nan)),
    ('tr2', GenderEncoder(**settings_file.gender_encoder)),
    ('tr3', AbsentPresentEncoder(**settings_file.absent_present_encoder)),
    ("tr4", CategoricalEncoder(**settings_file.categorical_encoder)),
    ('tr7', LABEncoder(**settings_file.lab_encoder)),
    ('OrdinalEncoder', OrdinalEncoder(**settings_file.ordinal_encoder)),
    ('ComputeAJCC', ComputeAJCC(**settings_file.compute_ajcc)),
    ('tr5', ExponentialTransformer(**settings_file.exponential_transformer)),
    ('RenameLabValues', RenameLabValues(**settings_file.rename_lab_values)),

    ('KeepColumns', KeepColumns(**settings_file.keep_cols)),
    ('CustomImputer', CustomImputer(strategy='mean')),
    #('CustomScaler', CustomScaler()),
    #('GBSA', GradientBoostingSurvivalAnalysis(loss='coxph'))
    #('CPH', CoxPHSurvivalAnalysis())
    #  ('model', FastSurvivalSVM(random_state=42))
    ('xgb', RandomSurvivalForest(random_state=42))
    ])


X_train, X_test, y_train, y_test = train_test_split(X.copy(), np.zeros(len(X)), test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test.copy(), np.zeros(len(X_test)), test_size=0.5, random_state=1)


y_train_list = get_target(X_train)
# y_val   = get_target(X_val)
y_test  = get_target(X_test)

y_train_list = Surv.from_dataframe(*y_train_list.columns, y_train_list)
#y_val = Surv.from_dataframe(*y_val.columns, y_val)
y_test = Surv.from_dataframe(*y_test.columns, y_test)

wsi_dict = {'train': [], 'val' : [], 'test': []}
#y_train = (y_train[1],y_train[0])

# Transform the data so we can use it
pipe.fit(X_train.copy(), y_train_list.copy())
#  X_val_c   = pipe.score(X_val.copy(), y_val)
X_test_c   = pipe.score(X_test.copy(), y_test)
total_c_index.append(X_test_c)

X_test_t = pipe[:-1].transform(X_test)
X_train_t = pipe[:-1].transform(X_train)

y_hat   = pipe[-1].predict_survival_function(X_test_t)

times   = np.arange(1,23)
preds   = np.asarray([[fn(t) for t in times] for fn in y_hat])
total_ib_score.append(integrated_brier_score(y_test, y_test, preds, times))
print(f'C-Index : {np.mean(total_c_index)} +- {np.std(total_c_index)}')
print(f'I - Brier-score : {np.mean(total_ib_score)} +- {np.std(total_ib_score)}')

C-Index : 0.7814756229346447 +- 0.0
I - Brier-score : 0.17157551408462574 +- 0.0


In [7]:
# First step: obtain Ho(t) from training data

from sksurv.nonparametric import nelson_aalen_estimator
times_train  = [x[1] for x in y_train_list]
events_train = [x[0] for x in y_train_list]

Ho_t_ = nelson_aalen_estimator(events_train, times_train)[1] # Unique times [0] ; CHF [1]
Ho_t_[18:]

array([0.53953808, 0.56290256, 0.57739532, 0.57739532, 0.60012259,
       0.62869402, 0.62869402, 0.62869402, 0.62869402, 0.62869402,
       0.62869402, 0.62869402, 0.62869402, 0.62869402, 0.62869402,
       0.62869402, 0.62869402, 1.62869402])

In [8]:
# We should add this to some sort of utils on the package
def fill_matrix(total_times, matrix, event_times):
    gl = []
    for time in total_times:
        if time in event_times:
            time_index = event_times.index(time)
            gl.append(matrix[time_index])
        else:
            gl.append(gl[-1])
    return gl


In [14]:
# Second step: Obtain the synthetic data
from survLime import survlime_tabular
num_pat = 500

columns = X_test_t.columns.tolist()
explainer = survlime_tabular.LimeTabularExplainer(X_train_t, feature_names=columns, class_names=None,
                                                   categorical_features=None, verbose=True, mode='regression', discretize_continuous=False)

synthetic_data = explainer.data_inverse(X_test_t.iloc[0], num_pat) # At [0] we have the data and at [1] the inverse, see what is this

In [15]:
# Third step: Obtain the prediction for the synthetic data 
H_i_j = pipe[-1].predict_cumulative_hazard_function(synthetic_data[1]) # X_test_t -> [371, num_features]
times_to_fill = list(set(times_train))
H_i_j[0]
H_i_j_wc = [fill_matrix(times_to_fill, x.y, list(x.x)) for x in H_i_j]

X does not have valid feature names, but RandomSurvivalForest was fitted with feature names


In [16]:
# Forth step: Compute the weights
from functools import partial
def kernel(d, kernel_width):
    return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2)) 
kernel_fn = partial(kernel, kernel_width=5)

# We need to do the line 362 from survlime_tabular (scale the data)
scaled_data = synthetic_data[0]
distances = sklearn.metrics.pairwise_distances(
            scaled_data,
            scaled_data[0].reshape(1,-1), # <-- Point of inquiry
            metric = 'euclidean').ravel()
weights = kernel_fn(distances)

In [17]:
import timeit
start_time = timeit.default_timer()
# code you want to evaluate
timeit.default_timer() - start_time


import cvxpy as cp
from math import log
epsilon = 0.00000001
n = 3 # For now we are only using 3 features
num_times = 34
b = cp.Variable(n)

cost = [weights[k]*cp.square((log(H_i_j_wc[k][j]+epsilon) - log(Ho_t_[j]+epsilon) - b @ scaled_data[k]))\
        *(times_to_fill[j+1]-times_to_fill[j]) for k in range(num_pat) for j in range(num_times)]
#cost = [weights[k]*cp.norm((log(H_i_j_wc[k][j]+epsilon) - log(Ho_t_[j]+epsilon) - b @ scaled_data[k]),'inf') \
#                                            for k in range(num_pat) for j in range(num_times)]
print(f'time creating the cost list {timeit.default_timer() - start_time}')

start_time = timeit.default_timer()
cost_sum = cp.sum(cost)

print(f'time summing the cost list {timeit.default_timer() - start_time}')
start_time = timeit.default_timer()

prob = cp.Problem(cp.Minimize(cost_sum))


opt_val = prob.solve(verbose=True)
print(f'time solving the problem {timeit.default_timer() - start_time}')
b.value

time creating the cost list 2.8493509613908827
time summing the cost list 2.9629415972158313
                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jun 27 12:13:05 PM: Your problem has 3 variables, 0 constraints, and 0 parameters.
(CVXPY) Jun 27 12:13:07 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 27 12:13:07 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 27 12:13:07 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 27 12:13:09 PM: Compiling problem (target solver=OSQP).
(CVX

array([-1.27623813,  1.10829642, -0.0129378 ])